# Testing the correlation between discrete traits

### Loading library

In [12]:
library("corHMM")
library("mclust")
library("stringr")
library("phytools")
library("qpcR")
library("secsse")
library("ggtree")
library("RColorBrewer")
library("ggplot2")
library("ggpmisc")
library("TreeTools")


Attachement du package : ‘TreeTools’


L'objet suivant est masqué depuis ‘package:ggtree’:

    MRCA


L'objet suivant est masqué depuis ‘package:phytools’:

    as.multiPhylo




### Loading data

In [7]:
df<-read.csv("Code_P_Muridae.tsv", sep="\t", header = TRUE) # omit sep ="\t" for .csv files
multi_phy<-read.tree("MURID_morpho_out_TreeDist.tre")

In [21]:
tr_lst<-c()
for (i in 1:length(multi_phy)){
    tr_lst[[i]]<-AddTip(multi_phy[[i]],
  where = which(phy$tip.label== "Hyomys_goliath"),
  label = "Hyomys_goliath2",
  edgeLength = 0.1,
  lengthBelow = 0.1
)
}

### Cleaning and preparing data

In [24]:
states_traits<-df[,c(1,2)]
colnames(states_traits)<-c("Species", "Cat")
states_traits[is.na(states_traits)]<-"?"

In [28]:
for (i in 1:1000){
    
phy<-multi_phy[[i]]

phy$tip.label<-paste(str_split_fixed(phy$tip.label, "_", 3)[,1], str_split_fixed(phy$tip.label, "_", 3)[,2], sep ="_")
phy<-AddTip(phy, where = which(phy$tip.label== "Hyomys_goliath"), label = "Hyomys_goliath2", edgeLength = 0.1, lengthBelow = 0.1)
phy$tip.label[phy$tip.label=='Taeromys_punicans']<-'Taeromys_arcuatus'
phy$tip.label[phy$tip.label=='Lenothrix_canus']<-'Lenothrix_cana'
phy$tip.label[phy$tip.label=='Hydromys_obi']<-'Hydromys_chrisogaster'
phy$tip.label[phy$tip.label=='Leptomys_elegans']<-'Leptomys_sp'
phy$tip.label[phy$tip.label=='Microhydromys_richardsoni']<-'Microhydromys_sp'
phy$tip.label[phy$tip.label=='Mesembriomys_gouldii']<-'Mesembryomys_gouldii'
phy$tip.label[phy$tip.label=='Solomys_sapientis']<-'Solomys_sapentis'
phy$tip.label[phy$tip.label=='Paramelomys_levipes']<-'Paramelomys_sp'
phy$tip.label[phy$tip.label=='Haeromys_minahassaeZ23000']<-'Haeromys_minahassae'
phy$tip.label[phy$tip.label=='Thallomys_sp']<-'Thallomys_spce'
phy$tip.label[phy$tip.label=='Pithecheir_chim']<-'Pithecheir_melanurus'

### One rate models

### Model 1 : equal rates

pt_1_eq<-corHMM(phy, states_traits, rate.cat = 1, rate.mat=NULL, model = "ER", node.states = "marginal",
fixed.nodes=FALSE, p=NULL, root.p="yang", ip=NULL, nstarts=0, n.cores=5,
get.tip.states = FALSE, lewis.asc.bias = FALSE, collapse = TRUE, lower.bound = 1e-9,
upper.bound = 100, opts=NULL)

### Model 2 : all rates differ

pt_1_ard<-corHMM(phy, states_traits, rate.cat = 1, rate.mat=NULL, model = "ARD", node.states = "marginal",
fixed.nodes=FALSE, p=NULL, root.p="yang", ip=NULL, nstarts=0, n.cores=5,
get.tip.states = FALSE, lewis.asc.bias = FALSE, collapse = TRUE, lower.bound = 1e-9,
upper.bound = 100, opts=NULL)

### Model 1 : equal rates

pt_2_eq<-corHMM(phy, states_traits, rate.cat = 2, rate.mat=NULL, model = "ER", node.states = "marginal",
fixed.nodes=FALSE, p=NULL, root.p="yang", ip=NULL, nstarts=0, n.cores=5,
get.tip.states = FALSE, lewis.asc.bias = FALSE, collapse = TRUE, lower.bound = 1e-9,
upper.bound = 100, opts=NULL)

### Model 2 : all rates differ

pt_2_ard<-corHMM(phy, states_traits, rate.cat = 2, rate.mat=NULL, model = "ARD", node.states = "marginal",
fixed.nodes=FALSE, p=NULL, root.p="yang", ip=NULL, nstarts=0, n.cores=5,
get.tip.states = FALSE, lewis.asc.bias = FALSE, collapse = TRUE, lower.bound = 1e-9,
upper.bound = 100, opts=NULL)

### Model 1 : equal rates

pt_3_eq<-corHMM(phy, states_traits, rate.cat = 3, rate.mat=NULL, model = "ER", node.states = "marginal",
fixed.nodes=FALSE, p=NULL, root.p="yang", ip=NULL, nstarts=0, n.cores=5,
get.tip.states = FALSE, lewis.asc.bias = FALSE, collapse = TRUE, lower.bound = 1e-9,
upper.bound = 100, opts=NULL)

### Model 2 : all rates differ

pt_3_ard<-corHMM(phy, states_traits, rate.cat = 3, rate.mat=NULL, model = "ARD", node.states = "marginal",
fixed.nodes=FALSE, p=NULL, root.p="yang", ip=NULL, nstarts=0, n.cores=5,
get.tip.states = FALSE, lewis.asc.bias = FALSE, collapse = TRUE, lower.bound = 1e-9,
upper.bound = 100, opts=NULL)

df_trait<-data.frame(cbind(c(pt_1_eq$AICc,  pt_1_ard$AICc,
                          pt_2_eq$AICc, pt_2_ard$AICc,
                          pt_3_eq$AICc,  pt_3_ard$AICc),
                akaike.weights(c(pt_1_eq$AICc, pt_1_ard$AICc,
                          pt_2_eq$AICc, pt_2_ard$AICc,
                          pt_3_eq$AICc,  pt_3_ard$AICc))$deltaAIC,
                akaike.weights(c(pt_1_eq$AICc, pt_1_ard$AICc,
                          pt_2_eq$AICc, pt_2_ard$AICc, 
                          pt_3_eq$AICc, pt_3_ard$AICc))$weights,
                c(
(max(as.vector(pt_1_eq$index.mat)[!is.na(as.vector(pt_1_eq$index.mat))])), (max(as.vector(pt_1_ard$index.mat)[!is.na(as.vector(pt_1_ard$index.mat))])),
(max(as.vector(pt_2_eq$index.mat)[!is.na(as.vector(pt_2_eq$index.mat))])), (max(as.vector(pt_1_ard$index.mat)[!is.na(as.vector(pt_1_ard$index.mat))])),
(max(as.vector(pt_3_eq$index.mat)[!is.na(as.vector(pt_3_eq$index.mat))])), (max(as.vector(pt_1_ard$index.mat)[!is.na(as.vector(pt_1_ard$index.mat))])))
                ))
rownames(df_trait)<-c("pt_1_eq", "pt_1_ard",
                          "pt_2_eq",  "pt_2_ard",
                          "pt_3_eq",  "pt_3_ard")
colnames(df_trait)<-c("AICc", "Delta_AICc", "AICcWt", "K_rates")
write.table(df_trait, paste("corHMM_output/model_replicate_", i, ".tsv"), sep ="\t")
}

State distribution in data:
States:	1	1&2	2	?	
Counts:	42	1	20	2	


Warning message in rate.cat.set.corHMM.JDB(phy = phy, data = input.data, rate.cat = rate.cat, :
“NAs introduits lors de la conversion automatique”
Warning message in rate.cat.set.corHMM.JDB(phy = phy, data = input.data, rate.cat = rate.cat, :
“NAs introduits lors de la conversion automatique”


Beginning thorough optimization search -- performing 0 random restarts 
Finished. Inferring ancestral states using marginal reconstruction. 


Warning message in rate.cat.set.corHMM.JDB(phy = phy, data = input.data, rate.cat = rate.cat, :
“NAs introduits lors de la conversion automatique”
Warning message in rate.cat.set.corHMM.JDB(phy = phy, data = input.data, rate.cat = rate.cat, :
“NAs introduits lors de la conversion automatique”
Warning message in corHMM(phy, states_traits, rate.cat = 1, rate.mat = NULL, model = "ER", :
“corHMM may have failed to optimize correctly, consider checking inputs and running again.”


State distribution in data:
States:	1	1&2	2	?	
Counts:	42	1	20	2	


...

Warning message in rate.cat.set.corHMM.JDB(phy = phy, data = input.data, rate.cat = rate.cat, :
“NAs introduits lors de la conversion automatique”
Warning message in rate.cat.set.corHMM.JDB(phy = phy, data = input.data, rate.cat = rate.cat, :
“NAs introduits lors de la conversion automatique”


Beginning thorough optimization search -- performing 0 random restarts 
Finished. Inferring ancestral states using marginal reconstruction. 


Warning message in rate.cat.set.corHMM.JDB(phy = phy, data = input.data, rate.cat = rate.cat, :
“NAs introduits lors de la conversion automatique”
Warning message in rate.cat.set.corHMM.JDB(phy = phy, data = input.data, rate.cat = rate.cat, :
“NAs introduits lors de la conversion automatique”
Warning message in corHMM(phy, states_traits, rate.cat = 1, rate.mat = NULL, model = "ARD", :
“corHMM may have failed to optimize correctly, consider checking inputs and running again.”


State distribution in data:
States:	1	1&2	2	?	
Counts:	42	1	20	2	


Warning message in rate.cat.set.corHMM.JDB(phy = phy, data = input.data, rate.cat = rate.cat, :
“NAs introduits lors de la conversion automatique”
Warning message in rate.cat.set.corHMM.JDB(phy = phy, data = input.data, rate.cat = rate.cat, :
“NAs introduits lors de la conversion automatique”


In [30]:
phy


Phylogenetic tree with 65 tips and 64 internal nodes.

Tip labels:
  Bandicota_indica, Rattus_macleari, Bunomys_chrysocomus, Taeromys_arcuatus, Taeromys_dominator, Rattus_nativitatis, ...

Rooted; includes branch lengths.